# Rigid Body 6-DoF System

The time derivative of a vector taken in the inertial frame, is equal to the time derivative of the vector taken in the body-fixed frame plus the cross product of the rotational rate and the vector.

\begin{align}
\frac{d^{(I)}\vec{v}}{dt} = \frac{d^{(b)}\vec{v}}{dt} + \vec{\omega} \times \vec{v}
\end{align}

where $I$ is the inertial frame and $b$ is a rotating frame, commonly referred to as the body-fixed frame in many mechanics applications.

Applying Newton's law to a system with mass $m$, velocity state $v_b = [u \; v \; w]^T$, force input $F_b = [F_x \; F_y \; F_z]^T$ and rotational rate $\omega_b = [p \; q \; r]^T$

\begin{align}
\dot{v}_b = \frac{F_b}{m} - \omega_b \times v_b
\end{align}

Subscripts indicate the frame of reference.

Finish this description...

In [ ]:
import control as ct
import numpy as np
from scipy.spatial.transform import Rotation as R


def dynamics(t, x, u, params):

    pos = x[:3] # inertial frame
    vel = x[3:6] # body frame
    quaternion = x[6:10] # (real, x, y, z)
    omega = x[10:]    

    force = u[:3]  
    moments = u[3:]

    # Position derivatives
    quat_body_to_inertial = np.roll(quaternion, -1) # Rotation package expects quat in (x, y, z, real) format
    R_body_to_inertial = R.from_quat(quat_body_to_inertial)
    ddt_pos = R_body_to_inertial.apply(vel)

    # Velocity derivatives
    ddt_vel = force / params['mass'] - np.cross(omega, vel)

    # Quaternion derivatives
    Q = np.array([[0, -omega[0], -omega[1], -omega[2]],
                [omega[0], 0, omega[2], -omega[1]],
                [omega[1], -omega[2], 0, omega[0]],
                [omega[2], omega[1], -omega[0], 0]])
    
    ddt_quaternion = .5*Q @ quaternion

    # Angular rate derivatives
    ddt_omega = np.linalg.inv(params['inertia']) @ (moments - np.cross(omega, params['inertia'] @ omega))
    
    return np.hstack([ddt_pos, ddt_vel, ddt_quaternion, ddt_omega])

def outputs(t, x, u, p):
    return x


x0 = np.array([initial_state[key] for key in [
    'x', 'y', 'z', 'u', 'v', 'w', 'qw', 'qx', 'qy', 'qz', 'p', 'q', 'r'
]])

states = ['x', 'y', 'z', 'u', 'v', 'w', 'qw', 'qx', 'qy', 'qz', 'p', 'q', 'r']
inputs = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']

rigid_body_6dof = ct.nlsys(dynamics, outfcn=outputs, states=states, inputs=inputs, outputs=states, name='rigid_body_6dof')

# References